In [1]:
# Here we test tfedlrn/collaborator/pytorchmodels/pytorch2dunet_NIfTY.py
# to determine if the data is getting loaded and preprocessed correctly

In [2]:
import sys
sys.path.append('/home/edwardsb/.local/share/virtualenvs/tfedlearn-cvKTHQG4/lib/python3.5/site-packages/')

In [3]:
from tfedlrn.collaborator.pytorchmodels.pytorch2dunet_NIfTY \
  import PyTorch2DUNetNIfTY as NIfTYNet

In [4]:
import tfedlrn.datasets as datasets

In [5]:
import numpy as np
import pickle
import os

In [6]:
model = NIfTYNet(device='cpu', shuffle_before_split=False, 
                 shuffle_before_epoch=True, percent_train=0.5)

NO SHUFFLE BEFORE SPLITTING INTO TRAIN/VAL!!!NO SHUFFLE BEFORE SPLITTING INTO TRAIN/VAL!!!NO SHUFFLE BEFORE SPLITTING INTO TRAIN/VAL!!!NO SHUFFLE BEFORE SPLITTING INTO TRAIN/VAL!!!NO SHUFFLE BEFORE SPLITTING INTO TRAIN/VAL!!!NO SHUFFLE BEFORE SPLITTING INTO TRAIN/VAL!!!NO SHUFFLE BEFORE SPLITTING INTO TRAIN/VAL!!!NO SHUFFLE BEFORE SPLITTING INTO TRAIN/VAL!!!NO SHUFFLE BEFORE SPLITTING INTO TRAIN/VAL!!!NO SHUFFLE BEFORE SPLITTING INTO TRAIN/VAL!!!
shapes imgs_all, msks_all: (155, 128, 128, 1), (155, 128, 128, 1)
shapes imgs_all, msks_all: (155, 128, 128, 1), (155, 128, 128, 1)
shapes imgs_all, msks_all: (155, 128, 128, 1), (155, 128, 128, 1)
shapes imgs_all, msks_all: (155, 128, 128, 1), (155, 128, 128, 1)
shapes imgs_all, msks_all: (155, 128, 128, 1), (155, 128, 128, 1)
shapes imgs_all, msks_all: (155, 128, 128, 1), (155, 128, 128, 1)
shapes imgs_all, msks_all: (155, 128, 128, 1), (155, 128, 128, 1)
shapes imgs_all, msks_all: (155, 128, 128, 1), (155, 128, 128, 1)
shapes imgs_all, msks

In [7]:
dataset_dir = datasets._get_dataset_dir()

In [8]:
imgs_inst0 = np.transpose(np.load('/raid/datasets/BraTS17/imgs_inst0.npy'), [0,3, 1, 2])
msks_inst0 = np.transpose(np.load('/raid/datasets/BraTS17/msks_inst0.npy'), [0,3, 1, 2])

In [11]:
results = []
for idx, (img, msk) in enumerate(model.train_loader):
    img = img.data.numpy
    msk = msk.data.numpy
    match = (np.all(img==imgs_inst0[idx]) and np.all(msk==msks_inst0[idx]))
    results.append(match)
final_result = np.all(results)
print("We succeeded.: {}".format(final_result))

We succeeded.: False


In [ ]:
# testing to see that the path to get the appropriate indexed image 
# looks good at first glance

In [ ]:
idx_to_train_paths = model.idx_to_train_paths

In [ ]:
directories = os.listdir(os.path.join(dataset_dir, \
                  'BraTS17/MICCAI_BraTS17_Data_Training/HGG'))
directories[0:3]

In [ ]:
# the paths below should match the concept of enumerateing all image slices
# obtained by grabbing 155 axial slice 
# images from each directory in the list above
idx_to_train_paths[0], idx_to_train_paths[1], idx_to_train_paths[2], \
idx_to_train_paths[154], idx_to_train_paths[155], idx_to_train_paths[310]

In [ ]:
# each brain (155 slices) given by a directory (associated to an index) 
# in the list:directories (defined above)
# corresponds to the institution number provided by the list
# brain_to_inst(below) at the same index

In [ ]:
with open(os.path.join(dataset_dir, 'BraTS17/brain_number_to_institution.pkl') \
          ,'rb') as file:
    brain_to_inst = pickle.load(file)
    
# convert to int and shifting by one to be 0-indexed
brain_to_inst = [int(entry)-1 for entry in brain_to_inst]
brain_to_inst[0:5]

In [ ]:
from shutil import copytree
# code for copying directories associated to a particular brain into
# a new parent directory associated to the institution from which that 
# brain belongs. This way, we get a new by_institution separation of the
# data, but this time with NIfTY files.
# In preparation, directories 0 - 9 need to be created under 
# '<data_dir>/BraTS17/by_inst_NIfTY'


for index, institution in enumerate(brain_to_inst):
    src = idx_to_train_paths[155*index]
    dir_name = src.split('/')[-1]
    dest = os.path.join(dataset_dir, 'BraTS17/by_institution_NIfTY/{}/{}'.
                        format(str(institution), dir_name))
    #print("copying from {} to {}".format(src, dest))
    copytree(src, dest)
    
    